# Some brief notes on linear regression

Linear regression is hard! [This paper](https://arxiv.org/pdf/1008.4686v1.pdf) is essential reading to see why. Some key points to note:

 * The response variable should always be variable with the smallest error/uncertainty in the values. OLS regression assumes the x values are known exactly and only models error in the y direction. If this condition isn't met you inference will be biased. If there are significant errors in both directions, consider a more advanced approach. <br><br>
 
 * Wherever possible, write down a **generative model** for your data (i.e. a likelihood function), rather than relying on an arbitrary **procedure** to perform your inference. Many commonly used procedures, such as minimising the sum of squared errors, have a direct interpretation as a particular likelihood function, but the latter is much more explicit about the assumptions being made and also more flexible. <br><br>
 
 * For OLS the key assumptions about normality, homoscedasticity, and lack of correlation apply to the **residuals**, not the variables themselves. The variables can have any distribution at all. If the residuals are significantly non-normal, and especially if they are heteroscedastic, transforming them (using e.g. Box-Cox) might help to satisfy the assumptions. <br><br>
 
 * For OLS multiple linear regression, it is **not necessary** (though it can be useful) to standardise the explanatory variables before performing the regression. (Standardising involves subtracting the mean and dividing by the standard deviation). It is often better to use the "raw" variable values (i.e. not standardised or transformed) if possible, as this will make the output easier to interpret. <br><br>
 
 * Standardising (or not) will make no difference to the significance estimates from the regression (see e.g. [here](http://stats.stackexchange.com/questions/29781/when-conducting-multiple-regression-when-should-you-center-your-predictor-varia)). The coefficient estimates will obviously change, though. If you're purely interested in looking for "significant" relationships between explanatory and response variables, it therefore shouldn't matter whether you standardise or not. Note, however, that if you variables are on very different scales the matrix calculations involved in performing the regression may become more difficult (and slower) to solve, and may even be numerically unstable. Standardisation may therefore be worthwhile, as long as you're willing to accept that it makes physical interpretation of the output a bit more difficult. <br><br>
 
 * In multiple linear regression, the coefficient for variable $X_j$ can be interpreted as, "the amount of change in $y$ as $X_j$ changes by one unit, *assuming all the other $X_i$ are kept constant.*". This means that you can "control" for other variables simply by including them in the model. For example, if you want to find the effect of sex on income, allowing for race, age, education etc., you simply need to include all the variables in the model e.g. `income ~ sex + age + race + education` and look at the coefficient for `sex`. See the two answers from whuber [here](http://stats.stackexchange.com/questions/17336/how-exactly-does-one-control-for-other-variables) and [here](http://stats.stackexchange.com/questions/167247/regression-analysis-with-confounding-variables-how-to-interpret-your-main-coeff). In other words, including other factors in the model is the same as controlling/allowing for them. <br><br>
 
 * As well as the significance of individual variables (i.e. the p-values), look at the overall probability from the F-test. This tests the overall hypothesis that at least one of your variable coefficients is different to zero i.e. that your model overall is better than just setting the intercept equal to the mean. Again, [this answer](http://stats.stackexchange.com/questions/167247/regression-analysis-with-confounding-variables-how-to-interpret-your-main-coeff) is useful. <br><br>
 
 * If the overall F-test is significant, but none of the individual variables are, this is often (but not exclusively) caused by collinearity in the predictors. See [here](http://stats.stackexchange.com/questions/3549/why-is-it-possible-to-get-significant-f-statistic-p-001-but-non-significant-r) and [here](http://stats.stackexchange.com/questions/14500/how-can-a-regression-be-significant-yet-all-predictors-be-non-significant) for details. This quote is especially useful:
 
    >  "*...the F-test has a low p-value (it is saying that the predictors together are highly significant in explaining the variation in the response variable). But the t-test for each predictor has a high p-value because after allowing for the effect of the other predictor there is not much left to explain*".

 * To **deal with collinearity**, consider using some form of penalised regression e.g. ridge, lasso or - even better - elastic net. Note that with all these approaches **you must standardise the predictors**, as otherwise the penalty term penalises the variables unequally. Standardising involves **centring** and **scaling** the data, usually this means **subtracting the mean** and **dividing by the standard deviation**, respectively (but see below for "**normalisation**" as an alternative to dividing by the standard deviation). Good implementations of these algorithms should standardise automatically (and then back-transform afterwards - read [this](http://se.mathworks.com/help/stats/ridge.html) carefully), but I'm not sure whether sklearn actually does this (?). See [here](https://stats.stackexchange.com/questions/186060/why-is-ridge-regression-giving-different-results-in-matlab-and-python/186070#186070) for details and a cautionary note about sklearn. Based on the sklearn documentation and some experimentation, I think the call signature for the sklearn functions should be interpreted as follows: <br><br>
 
   * `fit_intercept=True` is the equivalent of centring the data. This is done by default. <br><br>
   * `normalize=True` is an alternative scaling technique that is analagous to dividing by the standard deviation. Note that the default for this parameter is `False`.
 
 Most people seem to agree that you should ***always*** do some kind of standardisation before performing penalised regression, so the sklearn defaults are a bit misleading because they centre automatically, but don't scale. It looks as though the most sensible options are either: 
 
   1. Use `preprocessing.StandardScaler()` on the data first to subtract the mean and divide by the standard deviation. Then call the regularised regressor using `fit_intercept=False` (in which case the `normalize` argument is ignored). Or, <br><br>
   
   2. Pass the raw data to the regressor with `normalize=True` (and `fit_intercept=True`, assumed by default).
   
 These are two slightly different approaches to the same problem. Some differences are explained [here](http://stackoverflow.com/questions/32108179/linear-regression-normalization-vs-standardization) and [here](http://stats.stackexchange.com/questions/10289/whats-the-difference-between-normalization-and-standardization/10291#10291), but it seems to me that **standardising is much more usual** i.e. prefer option 1 (?). On the other hand, the sklearn documentation says
 
   > "*When the regressors are normalized, note that this makes the hyperparameters learnt more robust and almost independent of the number of samples. The same property is not valid for standardized data*".
 
 Actually, I'm not sure the above is correct - I can't get consistent results from sklearn's ridge regressor and plenty of others are questioning it's behaviour. The algorithm seems to give reasonable results with `fit_intercept=True` and `normalize=True`, but they're not the same as obtained from R. Use with caution! <br><br>
 
 * [This notebook](http://nbviewer.jupyter.org/urls/umich.box.com/shared/static/rg4sbfag376a5ffbhs47.ipynb) gives a nice comparison of the statsmodels elastic net implementation with the results from glmnet in R. They appear to give the same output, as long as you remember to standardise in Python first. Note that the following procedure is recommended: 
 
   1. Standardise all variables (including the response, although opinion on this elsewhere seems to differ). <br><br>
   
   2. Calculate any interaction terms (i.e. multiplying pairs of predictors together) and then standardise these as well. <br><br>
   
   3. Formulate the model omitting the intercept term (it's not necessary after centring the response). <br><br>
   
 * There's a good post and some links [here](http://stats.stackexchange.com/questions/154706/how-to-interpret-ridge-regression-plot) regarding how to interpret penalised regression output. Estimating parameter variance via bootstrapping (as I have done previously with sklearn) is recommended. Note that statsmodels gives summary output (p-values etc.) for elastic net regression, but I'm not sure whether this is meaningful (?). [This post](http://stats.stackexchange.com/questions/7225/estimating-r-squared-and-statistical-significance-from-penalized-regression-mode?rq=1) suggests it's not very meaningful (and also that bootstrap doesn't work either...). So it's not obvious how to use penalised regression outside of a "prediction-focused" context. <br><br>
 
 * Penalised regression has a natural Bayesian interpretation where the penalty term is represented by the prior. You can see this by writing down the log prior for a simple Bayesian regression model with Gaussian iid errors: the equation can be rearranged into the same form as ridge regression. Presumably this means that, if you intend to use mean zero priors within a Bayesian framework to shrink the parameters towards zero, you should **still standardise your predictors first** (?), just like for frequentist penalised regression. **Check!** It certainly seems to be the case that standardisation helps convergence of e.g the Metropolis algorithm (see [here](https://support.sas.com/rnd/app/stat/examples/BayesStd/new_example/index.html)), but I suspect this isn't a problem for more sophisticated algorithms. This suspicion seems to be confirmed [here](http://doingbayesiandataanalysis.blogspot.no/2013/01/bayesian-multiple-linear-regression.html), although lots of people still seem to standardise anyway. <br><br>
 
 * If data are subject to outliers, it's worth considering **robust regression**. Some frequentist options are available in statsmodels, but a nice place to start is with Bayesian robust regression. One simple option is to use a T-distribution rather than a Gaussian for the error model. The T-distribution has heavier tails and therefore penalises outliers less severely. See [here](https://pymc-devs.github.io/pymc3/notebooks/GLM-robust.html) for an example. A more sophisticated version, which includes outlier detection, is based on the example in [Hogg (2010)](https://arxiv.org/pdf/1008.4686v1.pdf) and is implemented in PyMC3 [here](https://pymc-devs.github.io/pymc3/notebooks/GLM-robust-with-outlier-detection.html). <br><br>
 
What a mess! The easiest option is probably to use sklearn's elastic net CV for a range of values for both alpha and lambda. Having found the best hyperparameters, you can use bootstrapping to estimate the variance in the parameter estimates. Or formulate your priors properly and do a fully Bayesian analysis, which is a bit more transparent.

In [44]:
from sklearn.linear_model import LinearRegression, Ridge
import pandas as pd
import numpy as np
from sklearn import preprocessing
from sklearn.linear_model import RidgeCV

# Data
df = pd.DataFrame(np.array([[1, 201, 592, 61, 9, -0.84],
                            [2, 244, 401, 25, 4, 0.31],
                            [3, 47, 583, 38, 11, 0.64],
                            [4, 287, 402, 15, 7, -0.27],
                            [5, 203, 495, 21, 5, -0.33],
                            [6, 58, 173, 15, 9, 0.67],
                            [7, 210, 479, 27, 4, -0.02],
                            [8, 202, 504, 14, 4, -0.05],
                            [9, 198, 510, 30, 11, -0.84],
                            [10, 158, 416, 16, 7, -0.69],
                            [11, 165, 393, 14, 5, 0.30],
                            [12, 201, 442, 25, 5, -0.46],
                            [13, 157, 317, 52, 5, -0.03],
                            [14, 131, 311, 16, 6, 0.50],
                            [15, 166, 400, 34, 6, 0.73],
                            [16, 160, 337, 31, 5, -0.52],
                            [17, 186, 423, 42, 9, 0.90],
                            [18, 125, 334, 26, 8, 0.40],
                            [19, 218, 533, 16, 6, -0.78],
                            [20, 146, 344, 22, 5, -0.56]]),
                   columns=['id','x1','y','x2','sigma_x','rho_xy'])

df.drop(['sigma_x','rho_xy'], axis=1, inplace=True)

## for convenience zero-base the 'id' and use as index
df['id'] = (df['id'] - 1).astype(int)
df.set_index('id', inplace=True)
df = df[['x1', 'x2', 'y']]

df.head()

,x1,x2,y
id,,,
0,201.0,61.0,592.0
1,244.0,25.0,401.0
2,47.0,38.0,583.0
3,287.0,15.0,402.0
4,203.0,21.0,495.0


In [61]:
y = df[['y']]
X = df[['x1', 'x2']]

y_std = preprocessing.scale(y)
y_cen = preprocessing.scale(y, with_std=False)
y_nor = preprocessing.normalize(y_cen, norm='l2')

X_std = preprocessing.scale(X)
X_cen = preprocessing.scale(X, with_std=False)
X_nor = preprocessing.normalize(X_cen, norm='l2')

print X_std.mean(axis=0), X_std.std(axis=0)
print X_nor.mean(axis=0), X_nor.std(axis=0)

[ -1.02695630e-16   4.44089210e-17] [ 1.  1.]
[ 0.03179728  0.02633593] [ 0.89585016  0.44243399]
[  1.55431223e-16] [ 1.]


In [70]:
model = Ridge(fit_intercept=True, normalize=True, alpha=0.5)
model.fit(X, y)

# Get param estimates
params = pd.Series(model.coef_[0], index=X.columns)
print params

x1    0.438795
x2    1.593449
dtype: float64


In [71]:
model = Ridge(fit_intercept=True, normalize=False, alpha=0.5)
model.fit(X_nor, y)
print model
# Get param estimates
params = pd.Series(model.coef_[0], index=df_X.columns)
print params

Ridge(alpha=0.5, copy_X=True, fit_intercept=True, max_iter=None,
   normalize=False, solver='auto', tol=0.001)
x1    61.571941
x2     9.061314
dtype: float64
